## Pair Programming Limpieza V

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

1. Es el momento de eliminar los nulos:
- Reemplazad los valores nulos del resto de las columnas categóricas por la moda, usando el método SimpleImputer.
    💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
os.getcwd()

'/mnt/c/Users/User/Desktop/ADALAB/MODULO2/promo-d-da-modulo2-sprint2-elisa-maitane'

In [3]:
os.chdir("limpieza")

In [4]:
df = pd.read_csv("../datos/bank-additional-full5.csv", index_col = 0)
df.head(2)

,age,job,marital,education,housing,loan,contact,duration,campaign,pdays,...,poutcome,"emp,var,rate","cons,price,idx","cons,conf,idx",euribor3m,"nr,employed",y,month_day_week,month,day_week
0,56.0,housemaid,married,basic 4y,No,No,telephone,261.0,1.0,999.0,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'
1,57.0,services,married,high school,No,No,telephone,149.0,1.0,999.0,...,NONEXISTENT,1.1,93.994,-36.4,4.857,"5191,0",no,"'may', 'mon'",'may','mon'


In [5]:
df_categoricas = df.select_dtypes(include='object').drop(columns=['education', 'housing', 'loan'], axis=1)
df_categoricas.head()

,job,marital,contact,poutcome,"nr,employed",y,month_day_week,month,day_week
0,housemaid,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
1,services,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
2,services,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
3,administrator,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'
4,services,married,telephone,NONEXISTENT,"5191,0",no,"'may', 'mon'",'may','mon'


In [6]:
from sklearn.impute import SimpleImputer

In [8]:
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[df_categoricas.columns])

In [10]:
df[df_categoricas.columns] = imputer.transform(df[df_categoricas.columns])

In [11]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
housing           0
loan              0
contact           0
duration          0
campaign          2
pdays             2
previous          2
poutcome          0
emp,var,rate      2
cons,price,idx    2
cons,conf,idx     2
euribor3m         2
nr,employed       0
y                 0
month_day_week    0
month             0
day_week          0
dtype: int64

- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [17]:
def eliminar_nulos_iterative(dataframe_completo):
    dataframe = dataframe_completo.select_dtypes(include = np.number)
    imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')
    imputer.fit(dataframe)
    numericas_trans = pd.DataFrame(imputer.transform(dataframe), columns = dataframe.columns)


    dataframe_completo.drop(columns = dataframe.columns, inplace= True)
    dataframe_completo.pd.concat([dataframe_completo, numericas_trans], axis = 1)
    
    return dataframe_completo

In [ ]:
df2 = 

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método KNN Imputer.

- Reemplazad los valores nulos de la columna age por la media de la edad usando el método Simple Imputer.

- ¿Podríais explicar qué diferencia hay entre estos últimos tres ejercicios?

2. Guardad el csv.